In [1]:
import numpy as np
import pythreejs
from moon import io as mio

lon=-11.36
lat=-43.31
side=5  # in degrees, sort of (on equirectangular projection :/)

range_lon = lon - side / 2, lon + side / 2
range_lat = lat - side / 2, lat + side / 2

dem_tycho = mio.square_cutout(lon, lat, side)

# pythreejs can't handle > 2 MB arrays?
n_drop = 1 # 5 # renders proper texture - why?
x = np.linspace(*range_lon, dem_tycho.shape[0])[::n_drop]
y = np.linspace(*range_lat, dem_tycho.shape[1])[::n_drop]
z = mio.get_tycho_cutout()[::n_drop, ::n_drop].astype(np.float32) / 1e4

ny, nx = z.shape

In [2]:
surf_g = pythreejs.SurfaceGeometry(z=list(z.flat), width=x.max()-x.min(), height=y.max()-y.min(),
                                   width_segments=nx-1, height_segments=ny-1)

surf = pythreejs.Mesh(geometry=surf_g, 
                      material=pythreejs.MeshLambertMaterial(map=pythreejs.height_texture(z, 'YlGnBu_r')))
surfgrid = pythreejs.SurfaceGrid(geometry=surf_g,
                                 material=pythreejs.LineBasicMaterial(color='black'))
scene = pythreejs.Scene(fog=pythreejs.Fog(near=2,far=5),
                        children=[surf, surfgrid,
                        pythreejs.AmbientLight(color='#777777')])
dlight = pythreejs.DirectionalLight(color='white', position=[3, 5, 1],
                                    intensity=0.6)
c = pythreejs.PerspectiveCamera(position=[0, 3, 3], up=[0, 0, 1], 
                                children=[dlight])
renderer = pythreejs.Renderer(camera=c, scene=scene,
                              width=900, height=600,
                              controls=[pythreejs.OrbitControls(controlling=c)])
renderer

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(3.0, 5.0,…

### TODOs:

1. Make a PR for broken numpy in SurfaceGeometry update method?
2. Fix the annoying non-square restriction - in particular, the texture doesn't work at all? 
3. Get the resterio geoslicing working and allow switching between crater names